# Analyzing Data in CSV Format Using SQL in a Notebook
In this beginner project, I will be analyzing the Electric Vehicle Population Data using SQL queries. The data is in CSV format. I will load this data into an in-memory SQLite database and perform my analysis.

In [1]:
# Importing Necessary Libraries
# pandas for data manipulation and sqlite3 to interact with the SQLite database
import pandas as pd
import sqlite3

#load the CSV data into a pandas DataFrame
df = pd.read_csv('C:/Users/PJ/Downloads/Electric_Vehicle_Population_Data.csv')

## Create an in-memory SQLite database and establish a connection to it.
I will create an in-memory SQLite database and establish a connection to it. This will allow us to interact with the data using SQL queries.

In [4]:
conn = sqlite3.connect(':memory:')

# write the DataFrame that I loaded from the CSV file into an SQL table in the SQLite database
df.to_sql('ev_data', conn, index=False)

135038

In [5]:
query = """
SELECT *
FROM ev_data
LIMIT 5;
"""
pd.read_sql_query(query, conn)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJ3E1EA0K,Thurston,Tumwater,WA,98512.0,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220.0,0.0,22.0,242565116,POINT (-122.91310169999997 47.01359260000004),PUGET SOUND ENERGY INC,5.306701e+10
1,1N4BZ1DV4N,Island,Clinton,WA,98236.0,2022,NISSAN,LEAF,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,10.0,183272785,POINT (-122.35936399999997 47.97965520000008),PUGET SOUND ENERGY INC,5.302997e+10
2,5YJ3E1EA0L,Snohomish,Snohomish,WA,98290.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,266.0,0.0,44.0,112552366,POINT (-122.09150499999998 47.91555500000004),PUGET SOUND ENERGY INC,5.306105e+10
3,5YJ3E1EBXL,King,Seattle,WA,98134.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,322.0,0.0,11.0,6336319,POINT (-122.32981499999994 47.579810000000066),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
4,5YJSA1CP0D,Snohomish,Edmonds,WA,98020.0,2013,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,69900.0,21.0,186212960,POINT (-122.37507 47.80807000000004),PUGET SOUND ENERGY INC,5.306105e+10


In [6]:
# Check for duplicates
query = """
SELECT COUNT(*) - COUNT(DISTINCT rowid) AS duplicates
FROM ev_data;
"""
pd.read_sql_query(query, conn)

,duplicates
0,0


In [7]:
# Check for missing values
query = """
SELECT
  SUM(CASE WHEN `VIN (1-10)` IS NULL OR `VIN (1-10)` = '' THEN 1 ELSE 0 END) AS vin_missing,
  SUM(CASE WHEN `County` IS NULL OR `County` = '' THEN 1 ELSE 0 END) AS county_missing,
  SUM(CASE WHEN `City` IS NULL OR `City` = '' THEN 1 ELSE 0 END) AS city_missing,
  SUM(CASE WHEN `State` IS NULL OR `State` = '' THEN 1 ELSE 0 END) AS state_missing,
  SUM(CASE WHEN `Postal Code` IS NULL OR `Postal Code` = '' THEN 1 ELSE 0 END) AS postal_code_missing,
  SUM(CASE WHEN `Model Year` IS NULL OR `Model Year` = '' THEN 1 ELSE 0 END) AS model_year_missing,
  SUM(CASE WHEN `Make` IS NULL OR `Make` = '' THEN 1 ELSE 0 END) AS make_missing,
  SUM(CASE WHEN `Model` IS NULL OR `Model` = '' THEN 1 ELSE 0 END) AS model_missing,
  SUM(CASE WHEN `Electric Vehicle Type` IS NULL OR `Electric Vehicle Type` = '' THEN 1 ELSE 0 END) AS ev_type_missing,
  SUM(CASE WHEN `Clean Alternative Fuel Vehicle (CAFV) Eligibility` IS NULL OR `Clean Alternative Fuel Vehicle (CAFV) Eligibility` = '' THEN 1 ELSE 0 END) AS cafv_eligibility_missing,
  SUM(CASE WHEN `Electric Range` IS NULL OR `Electric Range` = '' THEN 1 ELSE 0 END) AS electric_range_missing,
  SUM(CASE WHEN `Base MSRP` IS NULL OR `Base MSRP` = '' THEN 1 ELSE 0 END) AS base_msrp_missing,
  SUM(CASE WHEN `Legislative District` IS NULL OR `Legislative District` = '' THEN 1 ELSE 0 END) AS legislative_district_missing,
  SUM(CASE WHEN `DOL Vehicle ID` IS NULL OR `DOL Vehicle ID` = '' THEN 1 ELSE 0 END) AS dol_vehicle_id_missing,
  SUM(CASE WHEN `Vehicle Location` IS NULL OR `Vehicle Location` = '' THEN 1 ELSE 0 END) AS vehicle_location_missing,
  SUM(CASE WHEN `Electric Utility` IS NULL OR `Electric Utility` = '' THEN 1 ELSE 0 END) AS electric_utility_missing,
  SUM(CASE WHEN `2020 Census Tract` IS NULL OR `2020 Census Tract` = '' THEN 1 ELSE 0 END) AS census_tract_missing
FROM ev_data;

"""
pd.read_sql_query(query, conn)

,vin_missing,county_missing,city_missing,state_missing,postal_code_missing,model_year_missing,make_missing,model_missing,ev_type_missing,cafv_eligibility_missing,electric_range_missing,base_msrp_missing,legislative_district_missing,dol_vehicle_id_missing,vehicle_location_missing,electric_utility_missing,census_tract_missing
0,0,8,8,0,8,0,0,249,0,0,1,1,312,0,10,8,8


In [8]:
# fill missing values with mode
query = """
UPDATE ev_data
SET `County` = (SELECT `County` FROM (SELECT `County`, COUNT(*) AS count FROM ev_data GROUP BY `County` ORDER BY count DESC LIMIT 1) AS mode),
    `City` = (SELECT `City` FROM (SELECT `City`, COUNT(*) AS count FROM ev_data GROUP BY `City` ORDER BY count DESC LIMIT 1) AS mode),
    `Postal Code` = (SELECT `Postal Code` FROM (SELECT `Postal Code`, COUNT(*) AS count FROM ev_data GROUP BY `Postal Code` ORDER BY count DESC LIMIT 1) AS mode),
    `Electric Utility` = (SELECT `Electric Utility` FROM (SELECT `Electric Utility`, COUNT(*) AS count FROM ev_data GROUP BY `Electric Utility` ORDER BY count DESC LIMIT 1) AS mode),
    `2020 Census Tract` = (SELECT `2020 Census Tract` FROM (SELECT `2020 Census Tract`, COUNT(*) AS count FROM ev_data GROUP BY `2020 Census Tract` ORDER BY count DESC LIMIT 1) AS mode)
WHERE `County` IS NULL
   OR `City` IS NULL
   OR `Postal Code` IS NULL
   OR `Electric Utility` IS NULL
   OR `2020 Census Tract` IS NULL;
"""

# Execute the query
cursor = conn.cursor()
cursor.execute(query)

# Commit the changes
conn.commit()

In [9]:
# fill missing values with mean
query = """
UPDATE ev_data
SET `Electric Range` = (SELECT AVG(`Electric Range`) FROM ev_data),
    `Base MSRP` = (SELECT AVG(`Base MSRP`) FROM ev_data)
WHERE `Electric Range` IS NULL OR `Base MSRP` IS NULL;
"""
# Execute the query
cursor = conn.cursor()
cursor.execute(query)

# Commit the changes
conn.commit()

In [10]:
# fill missing values with 'Unknown'
query = """
UPDATE ev_data
SET `Model` = 'Unknown'
WHERE `Model` IS NULL;
"""
# Execute the query
cursor = conn.cursor()
cursor.execute(query)

# Commit the changes
conn.commit()

In [11]:
# fill missing values with 'Unknown'
query = """
UPDATE ev_data
SET `Legislative District` = 'Unknown'
WHERE `Legislative District` IS NULL;
"""
# Execute the query
cursor = conn.cursor()
cursor.execute(query)

# Commit the changes
conn.commit()

### Now that we have cleaned the data, let's proceed with the basic analysis of the data. 
#### Here are some analytical questions I try to answer using SQL:
- What is the distribution of electric vehicle types across different counties?
- Which make and model of electric vehicle is most popular in each county?
- What is the average electric range of vehicles by make and model?
- How many electric vehicles are eligible for the Clean Alternative Fuel Vehicle (CAFV) program?
- What is the distribution of electric vehicles by model year?
- Which electric utility serves the most electric vehicles?
- What is the average Base MSRP of electric vehicles by make and model?
- How many electric vehicles are there in each legislative district?
- What is the distribution of electric vehicles by postal code?
- Which city has the highest number of electric vehicles?

In [12]:
# What is the distribution of electric vehicle types across different counties?
query = """
SELECT County, `Electric Vehicle Type`, COUNT(*) as Count
FROM ev_data
GROUP BY County, `Electric Vehicle Type`
ORDER BY County, Count DESC;
"""
pd.read_sql_query(query, conn)

,County,Electric Vehicle Type,Count
0,Adams,Battery Electric Vehicle (BEV),28
1,Adams,Plug-in Hybrid Electric Vehicle (PHEV),12
2,Alameda,Battery Electric Vehicle (BEV),3
3,Alexandria,Battery Electric Vehicle (BEV),3
4,Alexandria,Plug-in Hybrid Electric Vehicle (PHEV),2
...,...,...,...
234,Wichita,Battery Electric Vehicle (BEV),2
235,Yakima,Battery Electric Vehicle (BEV),488
236,Yakima,Plug-in Hybrid Electric Vehicle (PHEV),247
237,Yavapai,Battery Electric Vehicle (BEV),1


In [13]:
# Which make and model of electric vehicle is most popular in each county?
# By counting the number of each make and model in each county and return the one with the highest count. This requires a bit more complex SQL, involving grouping, counting, and ordering, and then selecting the top row for each county.
query = """
WITH county_make_model_counts AS (
    SELECT County, Make, Model, COUNT(*) as Count
    FROM ev_data
    GROUP BY County, Make, Model
),
max_counts AS (
    SELECT County, MAX(Count) as MaxCount
    FROM county_make_model_counts
    GROUP BY County
)
SELECT cmm.County, cmm.Make, cmm.Model, cmm.Count
FROM county_make_model_counts cmm
JOIN max_counts mc ON cmm.County = mc.County AND cmm.Count = mc.MaxCount
ORDER BY cmm.County;
"""
pd.read_sql_query(query, conn)

,County,Make,Model,Count
0,Adams,TESLA,MODEL 3,7
1,Alameda,TESLA,MODEL 3,3
2,Alexandria,CHEVROLET,BOLT EV,1
3,Alexandria,JEEP,WRANGLER,1
4,Alexandria,KIA,SPORTAGE,1
...,...,...,...,...
210,Wichita,TESLA,MODEL 3,1
211,Wichita,TESLA,MODEL X,1
212,Yakima,TESLA,MODEL 3,146
213,Yavapai,TESLA,MODEL 3,1


In [14]:
# What is the average electric range of vehicles by make and model?
# By calculating the average electric range for each make and model.
query = """
SELECT Make, Model, AVG(`Electric Range`) as AvgElectricRange
FROM ev_data
GROUP BY Make, Model
ORDER BY AvgElectricRange DESC;
"""
pd.read_sql_query(query, conn)

,Make,Model,AvgElectricRange
0,HYUNDAI,KONA,258.000000
1,TESLA,ROADSTER,234.361702
2,JAGUAR,I-PACE,205.540541
3,TESLA,MODEL S,185.863241
4,TESLA,MODEL X,160.819245
...,...,...,...
121,TOYOTA,BZ4X,0.000000
122,VOLKSWAGEN,ID.4,0.000000
123,VOLVO,C40,0.000000
124,VOLVO,Unknown,0.000000


In [15]:
# How many electric vehicles are eligible for the Clean Alternative Fuel Vehicle (CAFV) program?
# By counting the number of vehicles that are eligible for the CAFV program
query = """
SELECT `Clean Alternative Fuel Vehicle (CAFV) Eligibility`, COUNT(*) as Count
FROM ev_data
GROUP BY `Clean Alternative Fuel Vehicle (CAFV) Eligibility`;
"""
pd.read_sql_query(query, conn)

,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Count
0,Clean Alternative Fuel Vehicle Eligible,61221
1,Eligibility unknown as battery range has not b...,56983
2,Not eligible due to low battery range,16834


In [16]:
# What is the distribution of electric vehicles by model year?
# By counting the number of vehicles for each model year
query = """
SELECT `Model Year`, COUNT(*) as Count
FROM ev_data
GROUP BY `Model Year`
ORDER BY `Model Year`;
"""
pd.read_sql_query(query, conn)

,Model Year,Count
0,1997,1
1,1998,1
2,1999,4
3,2000,9
4,2002,2
5,2003,1
6,2008,20
7,2010,23
8,2011,815
9,2012,1657


In [17]:
# Which electric utility serves the most electric vehicles?
# By counting the number of vehicles served by each electric utility
query = """
SELECT `Electric Utility`, COUNT(*) as Count
FROM ev_data
GROUP BY `Electric Utility`
ORDER BY Count DESC;
"""
pd.read_sql_query(query, conn)

,Electric Utility,Count
0,PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),49377
1,PUGET SOUND ENERGY INC,26606
2,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),24867
3,BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF C...,7816
4,BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,6041
...,...,...
71,CITY OF SEATTLE - (WA),1
72,BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF J...,1
73,BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF A...,1
74,BONNEVILLE POWER ADMINISTRATION||PENINSULA LIG...,1


In [18]:
# What is the average Base MSRP of electric vehicles by make and model?
# By calculating the average Base MSRP for each make and model
query = """
SELECT Make, Model, AVG(`Base MSRP`) as AvgBaseMSRP
FROM ev_data
GROUP BY Make, Model
ORDER BY AvgBaseMSRP DESC;
"""
pd.read_sql_query(query, conn)

,Make,Model,AvgBaseMSRP
0,PORSCHE,918,845000.000000
1,TESLA,ROADSTER,105553.191489
2,FISKER,KARMA,102000.000000
3,BMW,740E,90394.230769
4,CADILLAC,CT6,75095.000000
...,...,...,...
121,VOLVO,S60,0.000000
122,VOLVO,S90,0.000000
123,VOLVO,Unknown,0.000000
124,VOLVO,V60,0.000000


In [19]:
# How many electric vehicles are there in each legislative district?
# By counting the number of vehicles in each legislative district
query = """
SELECT `Legislative District`, COUNT(*) as Count
FROM ev_data
GROUP BY `Legislative District`
ORDER BY Count DESC;
"""
pd.read_sql_query(query, conn)

,Legislative District,Count
0,41.0,9035
1,45.0,8353
2,48.0,7637
3,36.0,5961
4,1.0,5746
5,5.0,5669
6,46.0,5433
7,43.0,5320
8,11.0,4575
9,37.0,4137


In [20]:
# What is the distribution of electric vehicles by postal code?
# By counting the number of vehicles in each postal code
query = """
SELECT `Postal Code`, COUNT(*) as Count
FROM ev_data
GROUP BY `Postal Code`
ORDER BY Count DESC;
"""
pd.read_sql_query(query, conn)

,Postal Code,Count
0,98052.0,3490
1,98033.0,2382
2,98012.0,2369
3,98004.0,2274
4,98006.0,2192
...,...,...
781,3804.0,1
782,2842.0,1
783,1824.0,1
784,1731.0,1


In [21]:
# Which city has the highest number of electric vehicles?
# By counting the number of vehicles in each city
query = """
SELECT City, COUNT(*) as Count
FROM ev_data
GROUP BY City
ORDER BY Count DESC;
"""
pd.read_sql_query(query, conn)

,City,Count
0,Seattle,23497
1,Bellevue,6960
2,Redmond,4965
3,Vancouver,4819
4,Kirkland,4201
...,...,...
646,Annapolis,1
647,Andrews Air Force Base,1
648,Altus,1
649,Alhambra,1
